In [9]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv(verbose=True,
            dotenv_path='./.env')
engine = create_engine(f'postgresql://{os.getenv("USERNAME")}:{os.getenv("PASSWORD")}@{os.getenv("HOST")}/{os.getenv("DATABASE")}', 
                       connect_args={'options': '-csearch_path={}'.format('de')})

In [10]:
visit_occurrence = pd.read_sql_table('visit_occurrence', engine)

# No.1
visit_occurrence 테이블은 병원에 방문한 환자들의 방문식별번호(id), 병원 방문
시작일자, 종료일자, 방문 타입(내원, 외래 등) 등 병원 방문과 관련된 정보를 포함하고 있습니다.  
내원일수는 환자가 요양기관을 방문하여 진료를 받은 일수이며, `내원일수 = 방문종료일자 - 방문시작일자 + 1` 으로 계산합니다. 모든 환자에 대해 총 내원일수를 구하고 총 내원일수의 최대값과 총 내원일수 최대값을 가지는
환자수를 찾는 쿼리를 작성합니다.
- 방문시작일자는 visit_start_date, 방문종료일자는 visit_end_date 를 사용합니다.

In [11]:
visit_occurrence.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41810 entries, 0 to 41809
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   visit_occurrence_id            41810 non-null  int64         
 1   person_id                      41810 non-null  int64         
 2   visit_concept_id               41810 non-null  int64         
 3   visit_start_date               41810 non-null  datetime64[ns]
 4   visit_start_datetime           41810 non-null  datetime64[ns]
 5   visit_end_date                 41810 non-null  datetime64[ns]
 6   visit_end_datetime             41810 non-null  datetime64[ns]
 7   visit_type_concept_id          41810 non-null  int64         
 8   provider_id                    0 non-null      object        
 9   care_site_id                   0 non-null      object        
 10  visit_source_value             41810 non-null  object        
 11  visit_source_co

`visit_start_date`와 `visit_end_date`는 모두 datetime 타입을 가지고 있기 때문에 손쉽게 며칠 간 내원했는지 계산할 수 있다.  
그리고 기존 `visit_occurrence` 테이블에 새로운 column으로 내원 일수를 저장한다.  
`value_counts()` 메소드를 통해 내원 일수의 최대값을 구할 수 있다. 동시에 최대 방문 일수를 가진 환자 수도 같이 찾을 수 있다.  
(3731일, 1명)


In [12]:
from datetime import timedelta
visit_occurrence['visit_dates'] = visit_occurrence['visit_end_date'] - visit_occurrence['visit_start_date'] + timedelta(days=1)

In [13]:
visit_occurrence['visit_dates'].value_counts().sort_index(ascending=False)

3731 days        1
714 days         1
705 days         1
696 days         1
695 days         1
             ...  
5 days          15
4 days          14
3 days          42
2 days        1765
1 days       39715
Name: visit_dates, Length: 89, dtype: int64

In [14]:
answer1 = visit_occurrence['visit_dates'].value_counts().sort_index(ascending=False).reset_index().rename(columns={'index': 'visit_dates', 'visit_dates': 'patients'})

In [15]:
answer1.head()

,visit_dates,patients
0,3731 days,1
1,714 days,1
2,705 days,1
3,696 days,1
4,695 days,1


In [16]:
# 내원 일수의 최대값을 가진 환자의 정보
visit_occurrence[visit_occurrence['visit_dates'] == timedelta(days=3731)]

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitted_from_concept_id,admitted_from_source_value,discharge_to_source_value,discharge_to_concept_id,preceding_visit_occurrence_id,visit_dates
18315,3508730,426322,9201,1989-05-28,1989-05-28 20:28:33,1999-08-14,1999-08-14 23:42:33,44818517,None,None,fcb4ac7a-eaeb-4b6a-895c-0f5a4b3abf56,0,0,None,0,None,54576630.0,3731 days
